In [1]:
%pip install statsmodels
%pip install mlxtend
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
import itertools
import random
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [3]:
#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

df_All_1o2 = pd.read_csv('3 Feature Selection/contData_all_Avg - 1o2.csv')
df_All_1o2 = pd.read_csv('3 Feature Selection/contData_all_Avg - 2o2.csv')

#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


In [4]:
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1o2, df_All_1o2], ignore_index=True)

print(df_All.head())

    425 %Al  Butanol   Decanol    Ethanol   Hexanol   Octanol  AYC55580  \
0  6.466835    8.950  3.271667  29.233325  5.862082  5.912500  9.708929   
1  6.479165    8.750  3.261668  29.233325  5.832917  5.870832  9.647859   
2  6.491500    8.525  3.310000  28.950000  5.767500  5.790000  9.592980   
3  6.503835    8.275  3.416667  28.383325  5.665832  5.670000  9.572995   
4  6.516165    8.025  3.523332  27.816675  5.564167  5.550000  9.586615   

    DI55102   DI55152   DI55580  ...    TI40050     TI52014     TI55013  \
0  1.016935  0.974092  0.991364  ...  74.705737  134.922750  204.003375   
1  1.017048  0.973535  0.991012  ...  74.042025  134.517750  203.384625   
2  1.017505  0.973474  0.990783  ...  73.626350  134.084875  202.217250   
3  1.017742  0.974004  0.990838  ...  73.203325  133.625625  201.412625   
4  1.017679  0.974445  0.991013  ...  72.900725  133.217250  201.149500   

      TI55014     TI55015     TI55016     TI55017     TI55021     TI55023  \
0  197.780625  193.19

In [5]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [6]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [7]:
# List of columns to exclude to run XGboost feature selection
exclude_columns = ['Octanol', 'Hexanol',
       'Ethanol', 'Decanol',
       
       'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
       'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'

       # 'FC55102', 'FC55152', 'LC55557', 'LC55568', 'TC55555',

       # '425 SAO Al', 'FFC55553', 'LC52572', 'LC90366',

       # 'FC42428', 'LC55553',

       # 'FC55009'
                   ]

# Create a new DataFrame without the excluded columnsd
df_All = df_All.drop(columns=exclude_columns)

In [8]:
df_All.columns

Index(['425 %Al', 'Butanol', 'AYC55580', 'DI55102', 'DI55152', 'DI55580',
       'FC42428', 'FC55003', 'FC55009', 'FC55102', 'FC55152', 'FC55552',
       'FC55555', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'FYC55553',
       'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
       'LC90368', 'PI55004', 'PI55020', 'PI55560', 'TC55552', 'TC55553',
       'TC55555', 'TC55566'],
      dtype='object')

In [9]:
# # Splitting into train and test
# X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
# y = df_All['Butanol']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import pandas as pd

def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42):
    results = []
    columns = [col for col in df_all.columns if col != target_column]
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        # Randomly order the remaining features
        remaining_features = [f for f in columns if f != feature]
        random.shuffle(remaining_features)

        # Create a new ordered list of features
        ordered_features = [feature] + remaining_features

        reordered_df = df_all[ordered_features + [target_column]]

        # Splitting into train and test for each permutation
        X = reordered_df.drop(target_column, axis=1)
        y = reordered_df[target_column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Create and fit the XGBoost model
        model = xgb.XGBRegressor(objective='reg:squarederror')
        model.fit(X_train, y_train)

        # Extract feature importances
        feature_importances = model.get_booster().get_score(importance_type="weight")

        # Store the result with the permutation order and feature importances
        results.append((ordered_features, feature_importances))

    return results

results = iterate_feature_rotations(df_All, 'Butanol')

# Convert results to a DataFrame
flattened_results = []
for ordered_features, importances in results:
    for feature, importance in importances.items():
        flattened_results.append({
            'Feature Rotation': ordered_features,
            'Feature': feature,
            'Importance': importance
        })

results_df = pd.DataFrame(flattened_results)


In [10]:
# Export the DataFrame to a CSV file
results_df.to_csv('3 Feature Selection/feature_rotation_results.csv', index=False)

print(results_df)

                                      Feature Rotation   Feature  Importance
0    [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   425 %Al       544.0
1    [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   LC55557       271.0
2    [425 %Al, LC55557, FFC55555, FC55555, TC55552,...  FFC55555       190.0
3    [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   FC55555       182.0
4    [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   TC55552       237.0
..                                                 ...       ...         ...
956  [TC55566, TC55552, FC55552, FC55555, PI55004, ...   DI55580        64.0
957  [TC55566, TC55552, FC55552, FC55555, PI55004, ...   LC52572       123.0
958  [TC55566, TC55552, FC55552, FC55555, PI55004, ...   LC90366       154.0
959  [TC55566, TC55552, FC55552, FC55555, PI55004, ...   425 %Al       184.0
960  [TC55566, TC55552, FC55552, FC55555, PI55004, ...   FC55152        86.0

[961 rows x 3 columns]


In [11]:
def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42, num_random_iterations=30):
    results = []
    columns = [col for col in df_all.columns if col != target_column]
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        for _ in range(num_random_iterations):
            # Randomly order the remaining features
            remaining_features = [f for f in columns if f != feature]
            random.shuffle(remaining_features)

            # Create a new ordered list of features
            ordered_features = [feature] + remaining_features

            reordered_df = df_all[ordered_features + [target_column]]

            # Splitting into train and test for each permutation
            X = reordered_df.drop(target_column, axis=1)
            y = reordered_df[target_column]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

            # Create and fit the XGBoost model
            model = xgb.XGBRegressor(objective='reg:squarederror')
            model.fit(X_train, y_train)

            # Extract feature importances
            feature_importances = model.get_booster().get_score(importance_type="weight")

            # Store the result with the permutation order and feature importances
            results.append((ordered_features, feature_importances))

    return results


results = iterate_feature_rotations(df_All, 'Butanol')

# Convert results to a DataFrame
flattened_results = []
for ordered_features, importances in results:
    for feature, importance in importances.items():
        flattened_results.append({
            'Feature Rotation': ordered_features,
            'Feature': feature,
            'Importance': importance
        })

results_df = pd.DataFrame(flattened_results)


In [12]:
print(results_df)

                                        Feature Rotation   Feature  Importance
0      [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   425 %Al       544.0
1      [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   LC55557       271.0
2      [425 %Al, LC55557, FFC55555, FC55555, TC55552,...  FFC55555       190.0
3      [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   FC55555       182.0
4      [425 %Al, LC55557, FFC55555, FC55555, TC55552,...   TC55552       237.0
...                                                  ...       ...         ...
28825  [TC55566, TC55553, FC55102, FC55152, PI55560, ...   LC55568       135.0
28826  [TC55566, TC55553, FC55102, FC55152, PI55560, ...  FFC55553       151.0
28827  [TC55566, TC55553, FC55102, FC55152, PI55560, ...   PI55020       141.0
28828  [TC55566, TC55553, FC55102, FC55152, PI55560, ...   FC55569       165.0
28829  [TC55566, TC55553, FC55102, FC55152, PI55560, ...   425 %Al       183.0

[28830 rows x 3 columns]


In [14]:
# Group by 'Feature' and calculate the average importance
average_importances = results_df.groupby('Feature')['Importance'].mean()

# Convert the Series to a DataFrame
average_importances_df = average_importances.reset_index()

# Rename the columns for clarity
average_importances_df.columns = ['Feature', 'Average Importance']

# Sort the DataFrame by 'Average Importance' in descending order
average_importances_df = average_importances_df.sort_values(by='Average Importance', ascending=False)

# Display or save the DataFrame
print(average_importances_df)
# Or save it to a CSV file
# average_importances_df.to_csv('average_feature_importances.csv', index=False)


     Feature  Average Importance
0    425 %Al          223.325806
27   TC55552          208.392473
6    FC55003          205.378495
12   FC55569          200.308602
22   LC90366          184.876344
13   FC55576          184.584946
14  FFC55553          180.894624
25   PI55020          174.434409
2    DI55102          171.945161
21   LC55568          165.893548
18   LC55553          164.563441
29   TC55555          164.163441
5    FC42428          163.369892
30   TC55566          162.705376
20   LC55557          159.604301
7    FC55009          159.296774
17   LC52572          158.851613
28   TC55553          158.660215
8    FC55102          158.011828
23   LC90368          156.638710
24   PI55004          154.949462
3    DI55152          151.033333
15  FFC55555          145.449462
19   LC55555          136.402151
11   FC55555          125.224731
9    FC55152          120.858065
26   PI55560          118.149462
1   AYC55580          116.624731
16  FYC55553          113.453763
10   FC555